## 0. DRIVE mount

In [63]:
# 구글 드라이브와 연동
from google.colab import drive
drive.mount("/content/gdrive")

# 연동 잘 되었는지 확인
!ls "gdrive/My Drive/Colab Notebooks/DACON/"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
k_fashion  LG_predict_complain


In [2]:
# zip 파일을 먼저 구글 클라우드에 업로드
# 압축파일을 세션 내 드라이브에 압축해제
!unzip "gdrive/My Drive/Colab Notebooks/DACON/LG_predict_complain/data.zip" -d 'data/'

Archive:  gdrive/My Drive/Colab Notebooks/DACON/LG_predict_complain/data.zip
  inflating: data/sample_submission.csv  
  inflating: data/test_err_data.csv  
  inflating: data/test_quality_data.csv  
  inflating: data/train_err_data.csv  
  inflating: data/train_problem_data.csv  
  inflating: data/train_quality_data.csv  


In [64]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)



In [65]:
# Data 저장할 경로
SAVEPATH = '/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain'

## 1. Data preprocess


### 1.1 Error data preprocess

#### 1.1.0 data load

In [62]:
#Data load
train_err  = pd.read_csv('/content/data/train_err_data.csv')
test_err = pd.read_csv('/content/data/test_err_data.csv')

#### 1.1.1 merge errtype data for user_id

In [ ]:
"""
에러 data user_id 별로 sum
"""
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

# train data
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000


id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
# error.shape

tmp_df = pd.DataFrame(error.tolist()).add_prefix('err')
tmp_df.to_csv(f"{SAVEPATH}/20210127_train_errtype_data.csv", index = False)
print(tmp_df.head())



100%|██████████| 16554663/16554663 [00:47<00:00, 348937.63it/s]


,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,err39,err40,err41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0


In [ ]:
# test data
test_user_number = 14999
test_user_id_min = 30000
test_user_id_max = 44998

test_id_error = test_err[['user_id','errtype']].values
test_error = np.zeros((test_user_number,42))
for person_idx, err in tqdm(test_id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_error[person_idx - test_user_id_min,err - 1] += 1
# error.shape

test_tmp_df = pd.DataFrame(test_error.tolist()).add_prefix('err')
test_tmp_df.to_csv(f"{SAVEPATH}/20210127_test_errtype_data.csv", index = False)
print(test_tmp_df.head())

#### 1.1.2 merge model_nm data for user_id

In [67]:
# train data

model_df = pd.get_dummies(train_err['model_nm'])
model_df['user_id'] = train_err['user_id']
model_df = model_df.groupby('user_id').sum().reset_index()
model_df.drop(['user_id'], axis=1,inplace=True)

# modelnm_all_col = list(model_df.columns)
# for col in modelnm_all_col:
#   model_df[col] = model_df[col].where(model_df[col] < 1, 1)

model_df.to_csv(f"{SAVEPATH}/20210203_train_modelnm_data.csv", index = False)
model_df

,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8
0,0.0,0.0,0.0,317.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2365.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,306.0,0.0,0.0,0.0,0.0,0.0,0.0
4,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,194.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
14997,826.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14998,155.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
# test data
test_model_df = pd.get_dummies(test_err['model_nm'])
test_model_col = list(test_model_df.columns)
test_model_df['user_id'] = test_err['user_id']
test_model_df = test_model_df.groupby('user_id').sum().reset_index()

# 43262 유저에 대한 정보가 없으므로 채워줘야 함
test_43262_user_data = [43262] + [0 for i in range(len(test_model_col))]
test_model_df.loc[14998]=test_43262_user_data
test_model_df = test_model_df.sort_values(by='user_id')
test_model_df = test_model_df.reset_index(drop=True)

test_model_df.drop(['user_id'], axis=1,inplace=True)

# modelnm_all_col = list(test_model_df.columns)
# for col in modelnm_all_col:
#   test_model_df[col] = test_model_df[col].where(test_model_df[col] < 1, 1)

test_model_df.to_csv(f"{SAVEPATH}/20210203_test_modelnm_data.csv", index = False)
test_model_df

,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8
0,0.0,2687.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0
2,941.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,371.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,623.0,258.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
14994,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14995,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,2233.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,24671.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 1.1.3 merge fwver data for user_id

In [ ]:
"""
groupby 적용시 메모리가 터질 위험이 있음
"""

'\ngroupby 적용시 메모리가 터질 위험이 있음\n'

In [ ]:
# train data
fwver_train_set = set(list(train_err['fwver'].unique()))
fwver_test_set = set(list(test_err['fwver'].unique()))
fwver_all_col = sorted(list(fwver_train_set | fwver_test_set))
push_train = list(fwver_test_set - fwver_train_set) # test에만 있는놈들, train에 0으로 채워줘야할 놈들
push_test = list(fwver_train_set - fwver_test_set) # train에만 있는놈들, test에 0으로 채워줘야할 놈들

fwver_df = pd.get_dummies(train_err['fwver'])
fwver_df[push_train] = 0
fwver_df['user_id'] = train_err['user_id']
fwver_df = fwver_df[['user_id'] +fwver_all_col]

sub_df_1 = fwver_df[['user_id']+fwver_all_col[:20]].groupby('user_id').sum().reset_index()

In [ ]:
sub_df_2 = fwver_df[['user_id']+fwver_all_col[20:]].groupby('user_id').sum().reset_index()
sub_df_2.drop(['user_id'], axis=1, inplace=True)

fwver_df = sub_df_1.join(sub_df_2)
#값이 1이상인 항목 전부 1로 맞춤
for col in fwver_all_col:
  fwver_df[col] = fwver_df[col].where(fwver_df[col] < 1, 1)
fwver_df.drop(['user_id'],axis=1, inplace=True)
fwver_df.to_csv(f"{SAVEPATH}/20210127_train_fwver_data.csv", index = False)
fwver_df

In [ ]:
# test data
fwver_train_set = set(list(train_err['fwver'].unique()))
fwver_test_set = set(list(test_err['fwver'].unique()))
fwver_all_col = sorted(list(fwver_train_set | fwver_test_set))
push_train = list(fwver_test_set - fwver_train_set) # test에만 있는놈들, train에 0으로 채워줘야할 놈들
push_test = list(fwver_train_set - fwver_test_set) # train에만 있는놈들, test에 0으로 채워줘야할 놈들

test_fwver_df = pd.get_dummies(test_err['fwver'])
test_fwver_df[push_test] = 0
test_fwver_df['user_id'] = test_err['user_id']
test_fwver_df = test_fwver_df[['user_id'] +fwver_all_col]

test_sub_df_1 = test_fwver_df[['user_id']+fwver_all_col[:20]].groupby('user_id').sum().reset_index()

In [ ]:
test_sub_df_2 = test_fwver_df[['user_id']+fwver_all_col[20:]].groupby('user_id').sum().reset_index()
test_sub_df_2.drop(['user_id'], axis=1, inplace=True)
test_fwver_df = test_sub_df_1.join(test_sub_df_2)
for col in fwver_all_col:
  test_fwver_df[col] = test_fwver_df[col].where(test_fwver_df[col] < 1, 1)
test_43262_user_data = [43262] + [0 for i in range(len(fwver_all_col))]
test_fwver_df.loc[14998]=test_43262_user_data
test_fwver_df = test_fwver_df.sort_values(by='user_id')
test_fwver_df = test_fwver_df.reset_index(drop=True)
test_fwver_df.drop(['user_id'],axis=1, inplace=True)
test_fwver_df.to_csv(f"{SAVEPATH}/20210127_test_fwver_data.csv", index = False)
test_fwver_df

#### 1.1.4 merge errorcode data for user_id

In [ ]:
train_user_max = 24999
train_user_min = 10000
train_user = 15000

tmp_list = list(range(train_user_min, train_user_max+1))
train_user_list = pd.DataFrame(tmp_list, columns=['user_id'])
train_user_list['problem'] = problem
p_list = list(train_user_list[train_user_list['problem'] == 1]['user_id'])
non_p_list = list(train_user_list[train_user_list['problem'] == 0]['user_id'])

print('SET preprocess')
# set 연산
p_list_set = set()
for li in tqdm(p_list):
  tmp_p_list = list(pd.DataFrame(train_err[train_err['user_id'] == li]['errcode'].value_counts()).index)
  p_list_set.update(tmp_p_list)

non_p_list_set = set()
for li2 in tqdm(non_p_list):
  non_tmp_p_list = list(pd.DataFrame(train_err[train_err['user_id'] == li2]['errcode'].value_counts()).index)
  non_p_list_set.update(non_tmp_p_list)

print('--------------')
print('--------------')
print('--------------')

print('DICT preprocess')
# dict 연산
p_list_dict = dict()
for li in tqdm(p_list):
  tmp_df = pd.DataFrame(train_err[train_err['user_id'] == li]['errcode'].value_counts())
  for d in list(tmp_df.index):
    p_list_dict[d] = p_list_dict.get(d,0) + tmp_df['errcode'][d]
# print(sorted(p_list_dict.items(), key= lambda x: x[1], reverse=True))


non_p_list_dict = dict()
for li in tqdm(non_p_list):
  non_tmp_df = pd.DataFrame(train_err[train_err['user_id'] == li]['errcode'].value_counts())
  for d in list(non_tmp_df.index):
    non_p_list_dict[d] = non_p_list_dict.get(d,0) + non_tmp_df['errcode'][d]
# print(sorted(non_p_list_dict.items(), key= lambda x: x[1], reverse=True))



  0%|          | 0/5000 [00:00<?, ?it/s]

SET preprocess


  0%|          | 5/5000 [00:00<01:59, 41.90it/s]

--------------
--------------
--------------
DICT preprocess


100%|██████████| 10000/10000 [04:00<00:00, 41.57it/s]


In [ ]:
sorted(p_list_dict.items(), key= lambda x: x[1], reverse=True)

[('1', 4075428),
 ('connection timeout', 1025237),
 ('0', 929364),
 ('B-A8002', 401056),
 ('14', 150728),
 ('80', 150201),
 ('79', 149407),
 ('active', 111323),
 ('2', 80781),
 ('84', 65521),
 ('85', 64549),
 ('connection fail to establish', 59263),
 ('3', 51345),
 ('NFANDROID2', 48787),
 ('standby', 48323),
 ('90', 34399),
 ('89', 33789),
 ('S-61001', 15933),
 ('95', 12473),
 ('4', 12094),
 ('94', 11737),
 ('13', 7016),
 ('6', 6571),
 ('5', 6524),
 ('8.0', 3860),
 ('Q-64002', 2917),
 ('78', 2644),
 ('81', 2233),
 ('connectionterminated by local host', 2153),
 ('UNKNOWN', 2084),
 ('86', 2045),
 ('terminate by peer user', 1990),
 ('V-21008', 1458),
 ('100', 1447),
 ('99', 1337),
 ('83', 1182),
 ('87', 679),
 ('H-51042', 674),
 ('21', 672),
 ('88', 655),
 ('82', 589),
 ('91', 536),
 ('92', 475),
 ('S-65002', 463),
 ('105', 449),
 ('J-30021', 426),
 ('93', 378),
 ('scanning timeout', 353),
 ('104', 343),
 ('77', 323),
 ('700001', 190),
 ('110', 166),
 ('V-21005', 144),
 ('76', 125),
 ('96

In [ ]:
sorted(non_p_list_dict.items(), key= lambda x: x[1], reverse=True)

[('1', 4831539),
 ('0', 1669759),
 ('connection timeout', 810363),
 ('B-A8002', 496807),
 ('80', 183817),
 ('79', 183015),
 ('14', 112849),
 ('active', 107915),
 ('2', 85601),
 ('84', 64355),
 ('85', 63195),
 ('standby', 62047),
 ('NFANDROID2', 55351),
 ('connection fail to establish', 41677),
 ('3', 39878),
 ('90', 31593),
 ('89', 31010),
 ('S-61001', 18698),
 ('95', 10979),
 ('94', 10381),
 ('Q-64002', 9042),
 ('4', 8155),
 ('8.0', 5457),
 ('13', 5024),
 ('78', 2384),
 ('81', 2106),
 ('86', 1978),
 ('connectionterminated by local host', 1858),
 ('UNKNOWN', 1830),
 ('100', 1155),
 ('S-65002', 1120),
 ('99', 1027),
 ('83', 905),
 ('B-51049', 870),
 ('6', 821),
 ('5', 811),
 ('87', 600),
 ('terminate by peer user', 533),
 ('88', 513),
 ('91', 465),
 ('82', 420),
 ('93', 387),
 ('H-51049', 378),
 ('H-51042', 371),
 ('J-30021', 339),
 ('92', 336),
 ('21', 310),
 ('77', 306),
 ('V-21008', 282),
 ('105', 270),
 ('700001', 212),
 ('110', 198),
 ('96', 167),
 ('104', 157),
 ('Q-64001', 144),


In [ ]:
inter_p_nonp = list(p_list_set.intersection(non_p_list_set))
val_list = []
for lis in inter_p_nonp:
  tmp_val = p_list_dict[lis]
  if tmp_val >= 100:
    # print(f"{lis} : {tmp_val}")
    val_list.append(lis)


In [ ]:
val_list

['87',
 '110',
 '91',
 'H-51042',
 'connectionterminated by local host',
 '96',
 'V-21008',
 '90',
 '76',
 'UNKNOWN',
 'Q-64002',
 '95',
 '78',
 'connection fail for LMP response timout',
 '92',
 '88',
 'active',
 '14',
 '83',
 '700001',
 '84',
 '80',
 '1',
 '105',
 '79',
 '6',
 'connection fail to establish',
 '81',
 'standby',
 '99',
 '0',
 '5',
 'J-30021',
 '93',
 '8.0',
 '86',
 'scanning timeout',
 'NFANDROID2',
 '82',
 'S-61001',
 'B-A8002',
 '4',
 '3',
 '21',
 '94',
 '85',
 'H-51046',
 'S-65002',
 '77',
 '100',
 '89',
 'V-21005',
 '2',
 '13',
 '104',
 'connection timeout',
 'terminate by peer user']

In [ ]:
# 문제인 사용자와 문제가 아닌 사용자의 에러로그 교집합
inter_p_nonp = list(p_list_set.intersection(non_p_list_set))

only_p_list = p_list_set.difference(non_p_list_set)
only_np_list = non_p_list_set.difference(p_list_set)

ignore_list = [] # 교집합 중에서 500 개 이상인 애들
n_ignore_list = [] # 교집합 중에서 500 개 미만 애들
val = 500
for ilist in inter_p_nonp:
  if p_list_dict[ilist] > val and non_p_list_dict[ilist] > val:
    ignore_list.append(ilist)
  else:
    n_ignore_list.append(ilist)
display(len(only_p_list))
print(sorted(only_p_list))
display(len(only_np_list))
print(sorted(only_np_list))
display(len(ignore_list))
print(sorted(ignore_list))
display(len(n_ignore_list))
print(sorted(n_ignore_list))


In [ ]:
only_p_list = ['10043', '10073', '10080', '10101', '10149', '10153', '10155', '10190', '10226', '10267', '10274', '102789', '10280', '10305', '103400', '10377', '10406', '10412', '10420', '10557', '10568', '105690', '10573', '10584', '10670', '10676', '10684', '10692', '10720', '10725', '10732', '10749', '10752', '10777', '10782', '10803', '10812', '108133', '10814', '10819', '10827', '10830', '10838', '10839', '10841', '10852', '10861', '10894', '10898', '1089908', '10902', '10934', '10938', '10945', '10949', '10955', '10961', '10972', '10995', '10997', '11007', '11014', '11043', '11063', '11068', '11108', '11113', '11146', '11151', '11251', '112910', '11295', '11324', '11396', '11399', '11534', '11650', '1166733', '11717', '11725', '1174404', '117471', '11751', '11906', '120256', '12093', '12125', '12265', '12308', '12320', '12370', '12491', '12687', '128798', '12952', '13013', '13130', '13171', '13240', '133231', '13339', '133496', '13474', '135370', '137526', '13924', '13987', '14044', '14154', '143915', '14424', '14642', '14739', '14757', '148188', '15040', '15169', '15250', '15303', '154638', '15612', '15649', '15700', '15715', '15720', '15734', '15748', '15771', '15790', '15800', '15850', '15874', '15900', '15920', '15939', '159695', '15979', '15989', '15999', '16012', '16015', '16017', '16027', '16050', '16079', '16123', '16142', '16158', '16163', '16241', '16253', '16399', '16883', '169623', '17073', '17233', '17456', '17502', '175629', '181089', '184457', '184780', '18770', '18836', '18989', '190999', '194956', '197102', '20', '20152', '20199', '20464', '20473', '20557', '20859', '20906', '20911', '20991', '21140', '21203', '21210', '21245', '21270', '21339', '21355', '214873', '21684', '21725', '22193', '2276093', '22838', '23252', '237720', '23866', '24001', '24122', '244599', '24569', '25322', '25372', '25412', '255847', '25667', '25796', '25807', '25957', '25962', '26057', '26118', '26145', '26166', '26213', '26243', '26246', '262579', '26347', '26369', '2638', '26414', '26569', '266296', '27198', '27690', '27741', '28380', '28506', '3001', '3020', '3021', '3026', '3028', '3030', '3036', '30376', '3039', '3041', '30415', '3047', '30531', '3058', '3061', '3064', '3067', '3068', '3078', '3082', '3091', '3101', '3105', '3111', '3113', '3117', '3124', '31309', '3133', '31361', '3140', '31418', '31460', '31523', '3154', '3156', '3159', '3163', '31632', '3167', '3173', '3174', '31794', '3180', '3184', '3189', '3194', '3201', '3203', '3208', '3209', '3249', '3250', '3268', '3269', '3270', '3272', '3277', '3294', '3296', '3301', '3303', '3313', '3317', '3319', '3320', '3336', '3342', '3349', '3350', '3360', '3365', '3370', '3371', '3375', '3376', '3377', '33785', '3380', '3382', '3383', '3384', '3390', '3391', '3400', '3401', '3415', '3431', '3434', '343440', '3439', '3441', '3445', '3454', '3455', '3458', '3459', '3463', '3464', '3468', '3469', '3471', '3482', '34835', '3486', '3487', '3494', '3496', '34999', '35065', '3507', '3508', '3512', '35124', '3513', '3517', '3520', '3521', '3522', '3527', '3533', '3534', '3535', '35370', '3543', '3550', '3552', '3558', '3565', '3567', '3571', '3575', '3578', '3579', '3580', '3581', '3582', '3583', '3585', '3592', '3593', '3598', '3602', '3610', '3614', '3617', '3618', '3621', '3623', '3624', '3625', '36252', '3627157', '36279', '36317', '36326', '36341', '3646', '36490', '3652', '36533', '3657', '36577', '3659', '36616', '36656', '3667', '3669', '3673', '3683', '36831', '3684', '3685', '3694', '3696', '36960', '3697', '3704', '3715', '3716', '3717', '3724', '3725', '3726', '3727', '3733', '3734', '3744', '3745', '3754', '3758', '3760', '3761', '3765', '3774', '37747', '3776', '3781', '3784', '3786', '3787', '3793', '3795', '3802', '3805', '3812', '3813', '38135', '3816', '3818', '3820', '3823', '3828', '38326', '3838', '3841', '3849', '3855', '3859', '3862', '3865', '3871', '3881', '3882', '3883', '3894', '3899', '3903', '3913', '3922', '3939', '39391', '3941', '3947', '3948', '3950', '3951', '3953', '3956', '3961', '39613', '3964', '3966', '3973', '3975', '3977', '3979', '3980', '39830', '3987', '3998', '40013', '4006', '4018', '4026', '4035', '4036', '4037', '4038', '4040', '4044', '4047', '4049', '4054', '4062', '4067', '4077', '4080', '4081', '4088', '4091', '4092', '4095', '4102', '41035', '4105', '41092', '4131', '4133', '4134', '41356', '41376', '4138', '4141', '4143', '4145', '4150', '41554', '4156', '41623', '4165', '4166', '4176', '41789', '4179', '4182', '4183', '4186', '4187', '41934', '4194', '4199', '42', '4210', '4211', '4224', '4225', '4231', '4235', '4244', '4254', '4255', '4259', '4261', '4262', '4268', '4270', '4278', '4280', '4285', '4288', '4291', '42925', '4313', '4314', '4334', '4337', '43400', '4345', '4348', '4354', '4357', '4358', '43650', '4389', '4399', '4404', '4424', '4430', '4435', '44380', '4439', '4441', '4443', '4449', '4459', '4463', '4475', '4488', '4502', '4513', '4514', '4521', '4523', '4539', '4545', '4550', '4563', '4567', '4570', '4572', '4575', '4576', '459578', '4596', '4608', '4618', '4621', '4622', '4629', '4630', '46411', '4645', '4654', '4661', '46679', '46800', '46925', '4696', '47', '4703', '4708', '4709', '4710', '4714', '47209', '4721', '4729', '4741', '4753', '4776', '4783', '4787', '4799', '4802', '4804', '4821', '48317', '4837', '4840', '4841', '4848', '4857', '4865', '4875823', '4886', '4893', '4894', '4903', '4904', '4905', '4912', '4916', '4922', '4930', '4935', '4943', '4946', '4956', '4963', '4975', '4978', '49781', '4982', '4986', '4994', '5002', '5003', '5007', '5014', '502630', '50275', '5041', '5046', '5057', '5090', '5102', '5113', '5123', '5132', '5138', '5144', '5146', '5157', '5159', '5160', '5166', '51670', '5168', '5175', '5176', '51803', '5185', '51858', '5188', '51964', '5199', '5205', '5208', '52094', '52103', '5215', '5226', '5248', '52593', '52668', '5270', '5292', '5295', '5311', '5341', '5342', '5344', '5352', '5362', '5372', '5374', '54043', '5406', '54094', '5418', '5422', '5429', '5437', '5438', '5441', '5457', '5458', '5465', '5466', '5468', '5475', '5476', '5480', '5482', '5503', '55163', '5526', '5535', '5538', '5539', '5540', '5559', '5564', '5576', '5577', '5578', '5586', '5587', '5593', '5602', '5604', '5605', '5607', '5615', '5617', '5619', '5621', '5627', '5629', '5642', '5645', '5649', '5658', '5659', '5664', '5666', '5667', '5669', '5670', '5671', '5676', '56789', '5682', '5686', '5689', '56896', '5693', '5705', '5714', '57247', '5725', '5729', '5730', '5731', '5733', '5741', '5742', '5743', '5746', '5751', '5752', '5760', '5761', '5770', '5776', '5782', '5788', '5790', '57938', '5797', '5817', '5819', '5824', '5825', '5828', '5833', '5836', '5837', '5839', '5845', '5849', '5853', '5861', '5867', '5869', '5873', '5880', '5884', '5891', '5903', '5909', '5911', '5922', '5925', '5936', '5938', '5945', '5946', '5952', '5962', '5967', '5968', '5972', '5975', '5976', '5980', '5985', '5986', '5994', '59951', '6005', '6008', '6010', '6013', '6014', '6015', '6020', '6021', '6023', '6032', '6034', '6038', '6039', '6053', '6064', '6065', '6074', '6088', '6091', '6093', '6121', '6122', '6123', '6142', '6152', '6175', '6176', '6179', '61828', '61852', '6192', '6193', '62', '6200', '6202', '6206', '62114', '62270', '6244', '62455', '62609', '6262', '6279', '6282', '6289', '6316', '6321', '6344', '6345', '6363', '6364', '6379', '64013', '6405', '6410', '6412', '64165', '6419', '6436', '6447', '6457', '6467', '6474', '6475', '6485', '6490', '6502', '6507', '6520', '6535', '6545', '6586', '65874', '6592', '6599', '6618', '6621', '6679', '6711', '6714', '6737', '6739', '6741', '6743', '67471', '6806', '6829', '6830', '6837', '6846', '6892', '6894', '6898', '6904', '6906', '69242', '6933', '6936', '6942', '6947', '6951', '6958', '7004', '7011', '7019', '7020', '7089', '7110', '7124', '7133', '7137', '7145', '7168', '7176', '7190', '7237', '72385', '72442', '72493', '72587', '72725', '72777', '7288', '7301', '73195', '733608', '7340', '7370', '7396', '7406', '7408', '7433', '7447', '7462', '74943', '7578', '7582', '7595', '7596', '7626', '7654', '7657', '7666', '7675', '7677', '7679', '7686', '7694', '7708', '7712', '7739', '7742', '7744', '7746', '7751', '7765', '7770', '7784', '7793', '7829', '7835', '7873', '7896', '7913', '7917', '7921', '7972', '7976', '79883', '8000', '8006', '8019', '8020', '8046', '8053', '8054', '8058', '8059', '8090', '8093', '8106', '8130', '8136', '8152', '8177', '8226', '82519', '8252', '8255', '82561', '8257', '8258', '8260', '8263', '82694', '8278', '82799', '8310', '8322', '83468', '8355', '8389', '8391', '8408', '8442', '8453', '8459', '8460', '8522', '8533', '8543', '85885', '8592', '8606', '8637', '8641', '8664', '8674', '8684', '8699', '8719', '8738', '8740', '87418', '87427', '8764', '8819', '8851', '8915', '892446', '8928', '8931', '8937', '8974', '9030', '9033', '9044', '9181', '9232', '9262', '9288', '9311', '9314', '9323', '9354', '9395', '9402', '9462', '9489', '9556', '956350', '9574', '9577', '9601', '9612', '9659', '9677', '968075', '9687', '9700', '9705', '9742', '9749', '9771', '97792', '97985', '9806', '9832', '9853', '9865', '9883', '9906', '9973', '9983', 'C-11087', 'C-13039', 'CM a', 'D-10011', 'En00406', 'En00409', 'J-20029', 'P_41007', 'U-81014', 'Y-00004', 'http']
only_np_list = ['10005', '10018', '10164', '10222', '10240', '10247', '10263', '10309', '10350', '10385', '103924', '10433', '10518', '10552', '10554', '10566', '10612', '10630', '10674', '10689', '10715', '10743', '10760', '10768', '10792', '10807', '10833', '10851', '10881', '10907', '10908', '10933', '10980', '11015', '11033', '11070', '11078', '11087', '11088', '11094', '11100', '11197', '11259', '113066', '11355', '11434', '11516', '11674', '11818', '118518', '11864', '11886', '11942', '11959', '12', '12026', '12099', '12103', '12120', '12202', '12498', '12650', '12785', '12977', '13162', '133555', '13394', '134588', '13708', '13759', '13995', '140055', '14708', '14920', '14942', '15589', '15857', '16098', '16126', '16153', '16195', '16231', '16352', '16860', '174669', '17722', '1858238', '18664', '18699', '205686', '20816', '20868', '20885', '20920', '20934', '21026', '21164', '21214', '21254', '21288', '21417', '21621', '21885', '230685', '23209', '23614', '23938', '24285', '25894', '25947', '25987', '25990', '25999', '26085', '26133', '26171', '26300', '26325', '27553', '3023', '3034', '3044', '3046', '3049', '3051', '3054', '3062', '3070', '3072', '3081', '3086', '3092', '3093', '3095', '3097', '3109', '3114', '31140', '3115', '31175', '3123', '3127', '3134', '31398', '3152', '3153', '3158', '3161', '3170', '3183', '3185', '3188', '3196', '3200', '3202', '3217', '3219', '3220', '32238', '3228', '3232', '3234', '3236', '3239', '3247', '3252', '3254', '3261', '3264', '3274', '3280', '3283', '3285', '3292', '3305', '3308', '3315', '3318', '3321', '3327', '3340', '3341', '3344', '3345', '3354', '3355', '3362', '3368', '3374', '3379', '3385', '3387', '3395', '3396', '3398', '33980', '3399', '3402', '3404', '3414', '3416', '3418', '3419', '3426', '3427', '3429', '3433', '3438', '3442', '3446', '3448', '3462', '3465', '3472', '3475', '3476', '3481', '3483', '3492', '3493', '3495', '3497', '3501', '3502', '3503', '3504', '3506', '3509', '3516', '3518', '3519', '3524', '3528', '3531', '3537', '3541', '3549', '3551', '3560', '3564', '3568', '3569', '3570', '3572', '3573', '3574', '3577', '3586', '3589', '3595', '3597', '3601', '3611', '3615', '36213', '3632', '3633', '36333', '36388', '3640', '3642', '3650', '36506', '36541', '3655', '3662', '3663', '3666', '3671', '3679', '3682', '3688', '3690', '3695', '37', '3707', '3712', '3714', '3719', '3720', '3732', '37411', '3747', '3752', '3753', '3755', '3756', '3762', '3763', '3768', '3772', '3777', '3778', '3788', '38', '3810', '3811', '3814', '3824', '3825', '3826', '3829', '3830', '38316', '3835', '3836', '3842', '3843', '3844', '3845', '3851', '3857', '38661', '3886', '3889', '38906', '3897', '3902', '3905', '3911', '3914', '3917', '3919', '3920', '3926', '3928', '3931', '3932', '3938', '3942', '3945', '3946', '3954', '3965', '3967', '3970', '3976', '3983', '4000', '4002', '4007', '4008', '40101', '4015', '4017', '4022', '4030', '4034', '4052', '4056', '4057', '40590', '4074', '4094', '4108', '4114', '4119', '4123', '4126', '41365', '41403', '4144', '4149', '4160', '41677', '4175', '41755', '41838', '4189', '4191', '419439', '4195', '4197', '4203', '4204', '4232', '4247', '4264', '4267', '4273', '4290', '4297', '4329', '4332', '4341', '43451', '4361', '4362', '4370', '4378', '4390', '4393', '4395', '4396', '4398', '4416', '4427', '4432', '4440', '4444', '4458', '4461', '4467', '4470', '44702', '4474', '4476', '4481', '4486', '4498', '4500', '4507', '4515', '4526', '4530', '4532', '4540', '4549', '4568866', '4590', '46', '4600', '4604', '4615', '4616', '4625', '4626', '4649', '4653', '46616', '46825', '4683', '46857', '4688', '4689', '46900', '4733', '4736', '4750', '4757', '4759', '4762', '4773', '4778', '4796', '4805', '4808', '4826', '4858', '4862', '4866', '4876', '4890', '4891', '4923', '4938', '4949', '4958', '4971', '4981', '4984', '4996', '5016', '5022', '5048', '5058', '5074', '5087', '5100', '5117', '5128', '51324', '5158', '51592', '51854', '5192', '51990', '52002', '5204', '5223', '5230', '5237', '5245', '5260', '5273', '527746', '5287', '5300', '5320', '5326', '5327', '5328', '5348', '5349', '5367', '5377', '5379', '5382', '5385', '5388', '5389', '5392', '5393', '5396', '5398', '5412', '5415', '5416', '5419', '5420', '5421', '5424', '5426', '5428', '5430', '5431', '5435', '5439', '5444', '5447', '5451', '5453', '5454', '5455', '5460', '5461', '5462', '5469', '5470', '5474', '5478', '5483', '5488', '5489', '5491', '5496', '5497', '5498', '5501', '5502', '5505', '5508', '5511', '5513', '5514', '5515', '55158', '5516', '5517', '5522', '5529', '5530', '5533', '5536', '5542', '5543', '5545', '5548', '5549', '5557', '5558', '5563', '5568', '5572', '5575', '5583', '5589', '5597', '5599', '5600', '5608', '5611', '5612', '5616', '5625', '5626', '5630', '5632', '5639', '5644', '5646', '5647', '5651', '5657', '5662', '5665', '5674', '56753', '5678', '5683', '56835', '5684', '5687', '5688', '56889', '5690', '5692', '56946', '5696', '56994', '5700', '5708', '5716', '5717', '5722', '5728', '57293', '5734', '5735', '5736', '5738', '5744', '5750', '5754', '5755', '5757', '5762', '5767', '5769', '5772', '5774', '5779', '5786', '5787', '5793', '5796', '5804', '5805', '5809', '5812', '5821', '5827', '5841', '5843', '5844', '5847', '5848', '5850', '5854', '5855', '5856', '5864', '5871', '5874', '5875', '5881', '5882', '5889', '5890', '5892', '5898', '5901', '5906', '5908', '5910', '5912', '5914', '5915', '5916', '5920', '5923', '5924', '5928', '5930', '5941', '5943', '5950', '5953', '5956', '5958', '5963', '5973', '5974', '5979', '5982', '5983', '5984', '5987', '5988', '5989', '5992', '5995', '5998', '5999', '6000', '6003', '6004', '6011', '6019', '6030', '6035', '6047', '6048', '6059', '6062', '6070', '6075', '6090', '6095', '6100', '6102', '6108', '6111', '6117', '6118', '6126', '6140', '6150', '6158', '61653', '6171', '6182', '6196', '61989', '6211', '6213', '6217', '6242', '6258', '6263', '6270', '6284', '6285', '6290', '6292', '6303', '6305', '6315', '6326', '6361', '6362', '6388', '6396', '6400', '6402', '6404', '6434', '6456', '6465', '6466', '6473', '65', '6501', '6506', '6513', '6528', '6532', '6554', '66', '6610', '6622', '6644', '6647', '6663', '6667', '6682', '6692', '6695', '6697', '66986', '67', '6702', '67245', '6725', '6726', '6751', '6758', '6759', '6767', '6778', '6796', '68', '6826', '6840', '6852', '6862', '6864', '69', '6901', '6920', '695552', '6961', '6977', '6993', '7005', '7040', '7052', '7053', '7092', '7094', '7100', '7107', '7138', '7187', '71924', '72038', '7228', '7243', '72438', '7248', '7250', '7274', '7316', '7318', '7321', '7327', '7339', '7352', '7358', '7428', '7442', '7454', '7474', '7475', '74770', '7497', '7535', '7549', '7551', '7553', '7563', '7633', '7663', '7670', '7698', '7704', '7756', '7771', '7795', '7797', '7821', '7837', '7872', '7904', '7910', '7918', '7926', '7927', '7933', '7962', '7968', '7984', '7989', '8022', '80477', '8056', '8061', '8077', '8087', '8091', '8105', '8127', '8156', '8172', '8186', '8200', '8228', '8239', '82415', '8271', '8339', '8350', '8356', '8372', '8397', '8398', '8414', '8462', '8473', '8488', '8494', '8515', '8549', '8610', '8645', '8673', '8731', '8733', '8755', '8760', '8771', '8793', '8811', '8840', '8843', '8855', '8918', '8940', '8944', '9028', '9032', '9069', '9088', '9131', '9185', '9197', '9199', '9214', '9229', '92862', '9301', '93129', '9334', '9364', '9398', '9473', '9506', '9519', '9551', '9557', '9585', '9598', '9619', '9638', '9683', '9774', '97772', '97883', '9843', '9855', '9859', '98606', '9900', '9934', '9938', '9971', 'C-11020', 'D-99999', 'M-51020', 'P-41001', 'PUBLIC_ERR', 'P_41001', 'S-64001', 'S-65', 'U-81000', 'U-82020', 'U-82023', 'V-21010']
ignore_list = ['0', '1', '100', '13', '14', '2', '3', '4', '5', '6', '78', '79', '8.0', '80', '81', '83', '84', '85', '86', '87', '88', '89', '90', '94', '95', '99', 'B-A8002', 'NFANDROID2', 'Q-64002', 'S-61001', 'UNKNOWN', 'active', 'connection fail to establish', 'connection timeout', 'connectionterminated by local host', 'standby', 'terminate by peer user']
n_ignore_list = ['-269', '-270', '0001', '101', '102', '103', '104', '105', '106', '107', '10721', '108', '109', '110', '15890', '15969', '16654', '20985', '21', '3002', '3003', '3004', '3005', '3006', '3007', '3008', '3009', '3010', '3011', '3012', '3013', '3014', '3015', '3016', '3017', '3018', '3019', '3022', '3024', '3025', '3027', '3029', '3031', '3032', '3033', '3035', '3038', '3040', '3042', '3043', '3045', '3048', '3050', '3052', '3053', '3055', '3056', '3057', '3059', '3060', '3063', '3065', '3066', '3069', '3071', '3073', '3074', '3075', '3076', '3077', '3079', '3080', '3083', '3084', '3085', '3087', '3088', '3089', '3090', '3094', '3096', '3098', '3099', '3102', '3103', '3104', '3106', '3107', '3108', '3110', '3112', '3116', '3118', '3120', '3121', '3122', '3125', '3126', '3128', '3129', '3130', '3131', '3132', '3136', '3137', '3138', '3139', '3141', '3142', '3143', '3144', '3145', '3146', '3147', '3148', '3149', '3150', '3151', '3155', '3157', '3160', '3162', '3164', '3165', '3166', '3168', '3175', '3176', '3177', '3178', '3181', '3182', '3186', '3187', '3190', '3191', '3192', '3193', '3195', '3197', '3198', '3204', '3205', '3206', '3207', '3210', '3212', '3214', '3215', '3218', '3221', '3222', '3223', '3224', '3225', '3226', '3227', '3229', '3231', '3233', '3235', '3237', '3238', '3240', '3241', '3242', '3243', '3244', '3245', '3246', '3248', '3251', '3253', '3256', '3257', '3258', '3259', '3260', '3262', '3263', '3265', '3266', '3267', '3271', '3273', '3275', '3276', '3278', '3279', '3281', '3282', '3284', '3286', '3287', '3288', '3289', '3290', '3291', '3293', '3295', '3298', '3299', '3300', '3302', '3304', '3306', '3307', '3309', '3310', '3311', '3312', '3314', '3316', '3322', '3323', '3324', '3325', '3326', '3328', '3329', '3330', '3332', '3337', '3338', '3339', '3343', '3346', '3347', '3348', '3353', '3356', '3357', '3358', '3359', '3361', '3363', '3364', '3366', '3367', '3372', '3373', '3378', '3386', '3388', '3389', '3392', '3393', '3394', '3403', '3405', '3407', '3408', '3409', '3410', '3411', '3412', '3413', '3420', '3421', '3423', '3424', '3425', '3428', '3430', '3432', '3435', '3436', '3437', '3440', '3444', '3456', '3457', '3460', '3461', '3466', '3470', '3473', '3477', '3478', '3479', '3480', '3484', '3485', '3488', '3489', '3490', '3491', '3498', '3499', '3500', '3505', '3510', '3511', '3514', '3523', '3525', '3526', '3532', '3536', '3538', '3539', '3542', '3544', '3545', '3546', '3548', '3553', '3555', '3557', '3559', '3566', '3576', '3584', '3587', '3588', '3590', '3591', '3594', '3599', '3600', '3606', '3608', '3609', '3616', '3619', '3626', '3627', '3630', '3638', '3639', '3641', '3643', '3644', '3645', '3648', '3649', '3653', '3654', '3658', '3664', '3672', '3674', '3677', '3681', '3691', '3693', '3699', '3703', '3706', '3709', '3710', '3713', '3718', '3721', '3722', '3723', '3737', '3741', '3743', '3746', '3750', '3767', '3769', '3773', '3783', '3806', '3809', '3821', '3822', '3827', '3839', '3847', '3854', '3875', '3876', '3877', '3895', '3910', '3912', '3925', '3929', '3933', '3943', '3959', '3971', '3974', '3984', '4012', '4019', '4045', '4058', '41', '4171', '4192', '4240', '4308', '4324', '4386', '4471', '4473', '4478', '4522', '4557', '4559', '4577', '4606', '4662', '4735', '4754', '4807', '4824', '4842', '4896', '4897', '4921', '5001', '5076', '5151', '5275', '5314', '5391', '5401', '5404', '5407', '5411', '5423', '5427', '5440', '5442', '5443', '5449', '5452', '5456', '5459', '5463', '5464', '5467', '5471', '5472', '5473', '5477', '5484', '5485', '5486', '5487', '5492', '5493', '5494', '5499', '5504', '5506', '5507', '5509', '5510', '5512', '5518', '5519', '5520', '5521', '5523', '5524', '5527', '5528', '5537', '5541', '5550', '5551', '5552', '5553', '5554', '5555', '5556', '5561', '5562', '5566', '5567', '5569', '5570', '5573', '5580', '5581', '5584', '5585', '5588', '5590', '5591', '5594', '5595', '5603', '5606', '5610', '5620', '5624', '5631', '5633', '5634', '5636', '5637', '5638', '5640', '5641', '5643', '5650', '5653', '5654', '5655', '5660', '5663', '5668', '5675', '5677', '5679', '5680', '5685', '5691', '5694', '5695', '5697', '5698', '5699', '5701', '5702', '5707', '5709', '5710', '5712', '5715', '5718', '5721', '5724', '5726', '5727', '5732', '5737', '5740', '5747', '5748', '5749', '5753', '5758', '5759', '5763', '5764', '5765', '5766', '5768', '5771', '5773', '5775', '5777', '5780', '5783', '5784', '5785', '5789', '5791', '5792', '5794', '5795', '5798', '5799', '5800', '5802', '5803', '5807', '5808', '5810', '5811', '5813', '5814', '5815', '5816', '5820', '5822', '5823', '5826', '5829', '5830', '5831', '5832', '5834', '5835', '5842', '5851', '5852', '5858', '5859', '5860', '5863', '5870', '5872', '5876', '5877', '5878', '5879', '5883', '5886', '5887', '5893', '5894', '5897', '5904', '5905', '5913', '5917', '5919', '5927', '5931', '5932', '5933', '5934', '5935', '5937', '5942', '5944', '5947', '5948', '5949', '5954', '5955', '5957', '5959', '5961', '5969', '5970', '5971', '5977', '5978', '5990', '5991', '5993', '5996', '5997', '60045', '6009', '6016', '6024', '6025', '6028', '6036', '6042', '6051', '6056', '6081', '6104', '6148', '6166', '6177', '6186', '6204', '6212', '6272', '6696', '6746', '70', '700001', '7090', '71', '72', '7236', '73', '74', '75', '76', '7656', '77', '82', '91', '92', '93', '96', '97', '98', 'B-51042', 'B-51049', 'C-11017', 'C-12032', 'C-13053', 'C-14014', 'E-59902', 'En00402', 'H-51042', 'H-51046', 'H-51048', 'H-51049', 'J-30010', 'J-30021', 'J-40011', 'L2CAP connection cancelled', 'M-51007', 'M-99999', 'P-41007', 'P-41007 ', 'P-41011', 'P-41011 ', 'P-44010', 'P-44010 ', 'PHONE_ERR', 'Q-64001', 'Q-73004', 'Q-73006', 'S-64000', 'S-64002', 'S-65002', 'U-81009', 'U-82004', 'U-82024', 'U-82026', 'V-21002', 'V-21003', 'V-21004', 'V-21005', 'V-21007', 'V-21008', 'Y-00005', 'Y-00008', 'connection fail for LMP response timout', 'scanning timeout']
display(len(only_p_list))
print(sorted(only_p_list))
display(len(only_np_list))
print(sorted(only_np_list))
display(len(ignore_list))
print(sorted(ignore_list))
display(len(n_ignore_list))
print(sorted(n_ignore_list))

1112

['10043', '10073', '10080', '10101', '10149', '10153', '10155', '10190', '10226', '10267', '10274', '102789', '10280', '10305', '103400', '10377', '10406', '10412', '10420', '10557', '10568', '105690', '10573', '10584', '10670', '10676', '10684', '10692', '10720', '10725', '10732', '10749', '10752', '10777', '10782', '10803', '10812', '108133', '10814', '10819', '10827', '10830', '10838', '10839', '10841', '10852', '10861', '10894', '10898', '1089908', '10902', '10934', '10938', '10945', '10949', '10955', '10961', '10972', '10995', '10997', '11007', '11014', '11043', '11063', '11068', '11108', '11113', '11146', '11151', '11251', '112910', '11295', '11324', '11396', '11399', '11534', '11650', '1166733', '11717', '11725', '1174404', '117471', '11751', '11906', '120256', '12093', '12125', '12265', '12308', '12320', '12370', '12491', '12687', '128798', '12952', '13013', '13130', '13171', '13240', '133231', '13339', '133496', '13474', '135370', '137526', '13924', '13987', '14044', '14154', 

938

['10005', '10018', '10164', '10222', '10240', '10247', '10263', '10309', '10350', '10385', '103924', '10433', '10518', '10552', '10554', '10566', '10612', '10630', '10674', '10689', '10715', '10743', '10760', '10768', '10792', '10807', '10833', '10851', '10881', '10907', '10908', '10933', '10980', '11015', '11033', '11070', '11078', '11087', '11088', '11094', '11100', '11197', '11259', '113066', '11355', '11434', '11516', '11674', '11818', '118518', '11864', '11886', '11942', '11959', '12', '12026', '12099', '12103', '12120', '12202', '12498', '12650', '12785', '12977', '13162', '133555', '13394', '134588', '13708', '13759', '13995', '140055', '14708', '14920', '14942', '15589', '15857', '16098', '16126', '16153', '16195', '16231', '16352', '16860', '174669', '17722', '1858238', '18664', '18699', '205686', '20816', '20868', '20885', '20920', '20934', '21026', '21164', '21214', '21254', '21288', '21417', '21621', '21885', '230685', '23209', '23614', '23938', '24285', '25894', '25947', '

37

['0', '1', '100', '13', '14', '2', '3', '4', '5', '6', '78', '79', '8.0', '80', '81', '83', '84', '85', '86', '87', '88', '89', '90', '94', '95', '99', 'B-A8002', 'NFANDROID2', 'Q-64002', 'S-61001', 'UNKNOWN', 'active', 'connection fail to establish', 'connection timeout', 'connectionterminated by local host', 'standby', 'terminate by peer user']


718

['-269', '-270', '0001', '101', '102', '103', '104', '105', '106', '107', '10721', '108', '109', '110', '15890', '15969', '16654', '20985', '21', '3002', '3003', '3004', '3005', '3006', '3007', '3008', '3009', '3010', '3011', '3012', '3013', '3014', '3015', '3016', '3017', '3018', '3019', '3022', '3024', '3025', '3027', '3029', '3031', '3032', '3033', '3035', '3038', '3040', '3042', '3043', '3045', '3048', '3050', '3052', '3053', '3055', '3056', '3057', '3059', '3060', '3063', '3065', '3066', '3069', '3071', '3073', '3074', '3075', '3076', '3077', '3079', '3080', '3083', '3084', '3085', '3087', '3088', '3089', '3090', '3094', '3096', '3098', '3099', '3102', '3103', '3104', '3106', '3107', '3108', '3110', '3112', '3116', '3118', '3120', '3121', '3122', '3125', '3126', '3128', '3129', '3130', '3131', '3132', '3136', '3137', '3138', '3139', '3141', '3142', '3143', '3144', '3145', '3146', '3147', '3148', '3149', '3150', '3151', '3155', '3157', '3160', '3162', '3164', '3165', '3166', '3168'

In [ ]:
# def change_val(tmp_val):
#   if tmp_val in only_p_list:
#     return 'A'
#   if tmp_val in only_np_list:
#     return  'B'
#   if tmp_val in ignore_list:
#     return  'C'
#   if tmp_val in n_ignore_list:
#     return  'D'
#   else:
#     return  'UNKNOWN'
def change_val(tmp_val):
  if tmp_val in val_list:
    return 'A'
  else:
    return 'B'

print("train task....")
train_err['errcode_change'] = train_err['errcode'].map(lambda x : change_val(x))
train_err = train_err.join(pd.get_dummies(train_err['errcode_change']))
error_combine = pd.get_dummies(train_err['errcode_change'])
error_combine['user_id'] = train_err['user_id']
error_combine = error_combine[['user_id','A','B']]
error_combine = error_combine.groupby('user_id').sum().reset_index()
print(error_combine.head())
error_combine.drop(['user_id'],axis=1, inplace=True)
error_combine.to_csv(f"{SAVEPATH}/20210131_train_errcode_data_2.csv", index = False)
print('save done')

print("-------------------")
print("-------------------")
print("-------------------")

print("test task....")
test_err['errcode_change'] = test_err['errcode'].map(lambda x : change_val(x))
test_err = test_err.join(pd.get_dummies(test_err['errcode_change']))
test_error_combine = pd.get_dummies(test_err['errcode_change'])
test_error_combine['user_id'] = test_err['user_id']
test_error_combine = test_error_combine[['user_id','A','B']]
test_error_combine = test_error_combine.groupby('user_id').sum().reset_index()
print(test_error_combine.head())

test_43262_user_data = [43262] + [0 for i in range(2)]
test_error_combine.loc[14998]=test_43262_user_data
test_error_combine = test_error_combine.sort_values(by='user_id')
test_error_combine = test_error_combine.reset_index(drop=True)
# print(test_error_combine)
test_error_combine.drop(['user_id'],axis=1, inplace=True)
test_error_combine.to_csv(f"{SAVEPATH}/20210131_test_errcode_data_2.csv", index = False)
print('save done')

train task....
   user_id       A    B
0    10000   317.0  0.0
1    10001  2365.0  0.0
2    10002   306.0  0.0
3    10003   304.0  2.0
4    10004   777.0  0.0
save done
-------------------
-------------------
-------------------
test task....
   user_id       A    B
0    30000  2750.0  0.0
1    30001   283.0  1.0
2    30002   941.0  0.0
3    30003   371.0  0.0
4    30004   880.0  1.0
save done


#### 1.1.5 merge datetime data for user_id

In [ ]:
def make_weekday_df(df, err_or_qual):
  """
  요일별(월(0)~일(6))로 데이터를 병합해주는 함수

  df = train or test 데이터프레임
  err_or_qual = 에러 데이터인 경우 'err', 퀄리티 데이터인 경우 'qual' 로 표기
  """

  df_datetime = df[['user_id','time']]

  # 주 = [월 화 수 목 금 토 일]
  print("create datetime...",end='')
  df_datetime['datetime'] = df_datetime['time'].apply(make_datetime)
  print('done!')
  # print("create month")
  # df_datetime['month'] = df_datetime['datetime'].map(lambda x : x.month)
  # print("create day")
  # df_datetime['day'] = df_datetime['datetime'].map(lambda x : x.day)
  # print("create hour")
  # df_datetime['hour'] = df_datetime.loc[:, 'datetime'].dt.time
  print("create weekday...",end='')
  df_datetime['weekday'] = df_datetime['datetime'].map(lambda x : x.weekday())
  # display(df_datetime)
  print('done!')

  #주만 따로 분배
  print("merge weekday...",end='')
  df_weekday = pd.get_dummies(df_datetime['weekday'])

  #주 별로 나눈 후 groupby sum
  df_weekday = pd.get_dummies(df_datetime['weekday']).add_prefix(f'{err_or_qual}_week_')
  df_weekday['user_id'] = df['user_id']
  df_weekday = df_weekday.groupby('user_id').sum().reset_index()
  print('done!')
  print(df_weekday)

  return df_weekday

In [ ]:
# train data
train_err_weekday = make_weekday_df(train_err, 'err')
train_err_weekday.drop(['user_id'],axis=1, inplace=True)
train_err_weekday.to_csv(f"{SAVEPATH}/20210201_train_err_weekday_data.csv", index = False)
print("save data")

# test data
test_err_weekday = make_weekday_df(test_err, 'err')

# test data 43262 유저 데이터 0으로 넣어주기
test_43262_user_data = [43262] + [0 for i in range(len(test_err_weekday.columns)-1)]
test_err_weekday.loc[14998]=test_43262_user_data
test_err_weekday = test_err_weekday.sort_values(by='user_id')
test_err_weekday = test_err_weekday.reset_index(drop=True)
test_err_weekday.drop(['user_id'],axis=1, inplace=True)
test_err_weekday.to_csv(f"{SAVEPATH}/20210201_test_err_weekday_data.csv", index = False)
print("save data")

create datetime...done!
create weekday...done!
merge weekday...done!
       user_id  err_week_0  err_week_1  ...  err_week_4  err_week_5  err_week_6
0        10000        37.0        41.0  ...        43.0        54.0        52.0
1        10001       164.0       138.0  ...      1534.0       117.0       139.0
2        10002        52.0        34.0  ...        34.0        42.0        58.0
3        10003        51.0        52.0  ...        37.0        22.0        57.0
4        10004        96.0       144.0  ...       171.0        84.0       108.0
...        ...         ...         ...  ...         ...         ...         ...
14995    24995        40.0         0.0  ...        54.0        39.0        34.0
14996    24996         4.0         0.0  ...         0.0         0.0         0.0
14997    24997       125.0       112.0  ...       109.0       118.0       133.0
14998    24998        23.0        47.0  ...        19.0        12.0        24.0
14999    24999        83.0       108.0  ...        

#### 1.1.6 merge datetime_time data for user_id

In [23]:
def make_hours_df(df, err_or_qual):
  """
  요일별(월(0)~일(6))로 데이터를 병합해주는 함수

  df = train or test 데이터프레임
  err_or_qual = 에러 데이터인 경우 'err', 퀄리티 데이터인 경우 'qual' 로 표기
  """

  df_datetime = df[['user_id','time']]

  # 주 = [월 화 수 목 금 토 일]
  print("create datetime...",end='')
  df_datetime['datetime'] = df_datetime['time'].apply(make_datetime)
  print('done!')
  print("create hours...",end='')
  df_datetime['hours'] = df_datetime['datetime'].map(lambda x : x.time().hour)
  # display(df_datetime)
  print('done!')

  #주만 따로 분배
  print("merge hours...",end='')
  df_hours = pd.get_dummies(df_datetime['hours'])

  #주 별로 나눈 후 groupby sum
  df_hours = pd.get_dummies(df_datetime['hours']).add_prefix(f'{err_or_qual}_hours_')
  df_hours['user_id'] = df['user_id']
  df_hours = df_hours.groupby('user_id').sum().reset_index()
  print('done!')
  print(df_hours)

  return df_hours

In [24]:
# train data
train_err_hours = make_hours_df(train_err, 'err')
train_err_hours

train_err_hours.drop(['user_id'],axis=1, inplace=True)
train_err_hours.to_csv(f"{SAVEPATH}/20210203_train_err_hours_data.csv", index = False)
print("save data")

# test data
test_err_hours = make_hours_df(test_err, 'err')

# test data 43262 유저 데이터 0으로 넣어주기
test_43262_user_data = [43262] + [0 for i in range(len(test_err_hours.columns)-1)]
test_err_hours.loc[14998]=test_43262_user_data
test_err_hours = test_err_hours.sort_values(by='user_id')
test_err_hours = test_err_hours.reset_index(drop=True)
test_err_hours.drop(['user_id'],axis=1, inplace=True)
test_err_hours.to_csv(f"{SAVEPATH}/20210203_test_err_hours_data.csv", index = False)
print("save data")

create datetime...done!
create hours...done!
merge hours...done!
       user_id  err_hours_0  ...  err_hours_22  err_hours_23
0        10000         33.0  ...           5.0           2.0
1        10001         62.0  ...          70.0          45.0
2        10002          4.0  ...          13.0          10.0
3        10003         32.0  ...           2.0           3.0
4        10004          7.0  ...          70.0          30.0
...        ...          ...  ...           ...           ...
14995    24995          9.0  ...          18.0          16.0
14996    24996          0.0  ...           0.0           0.0
14997    24997         21.0  ...          62.0          41.0
14998    24998          0.0  ...           0.0           0.0
14999    24999          1.0  ...           2.0           0.0

[15000 rows x 25 columns]
save data
create datetime...done!
create hours...done!
merge hours...done!
       user_id  err_hours_0  ...  err_hours_22  err_hours_23
0        30000         37.0  ...        

### 1.2 quality data preprocess

#### 1.2.1 change data type(object to float)

#### 1.2.0 data load

In [50]:
"""
퀄리티 data object to float
"""
#Data load
train_qual = pd.read_csv('/content/data/train_quality_data.csv')
test_qual = pd.read_csv('/content/data/test_quality_data.csv')


In [51]:

#Fillna
train_qual = train_qual.fillna(0)
display(train_qual.info())
test_qual = test_qual.fillna(0)
display(test_qual.info())

#change object to float
tmp_list = ['quality_0','quality_1','quality_2', 'quality_3', 'quality_4', 'quality_5',
 'quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']

print("train data change column type")
for li in tmp_list:
  if str(train_qual[li].dtypes) == 'object':
    print(f'column {li} is object. change object to float')
    train_qual[li] = train_qual[li].map(lambda x: str(x).replace(',',''))
    train_qual[li] = train_qual[li].astype(float)

print("done..")
display(train_qual.info())

print("--------------------------------------")
print("--------------------------------------")
print("--------------------------------------")

print("test data change column type")
for li in tmp_list:
  if str(test_qual[li].dtypes) == 'object':
    print(f'column {li} is object. change object to float')
    test_qual[li] = test_qual[li].map(lambda x: str(x).replace(',',''))
    test_qual[li] = test_qual[li].astype(float)
print("done..")
display(test_qual.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       828624 non-null  object 
 3   quality_0   828624 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   828624 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828624 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        747972 non-null  int64  
 1   user_id     747972 non-null  int64  
 2   fwver       747972 non-null  object 
 3   quality_0   747972 non-null  float64
 4   quality_1   747972 non-null  object 
 5   quality_2   747972 non-null  float64
 6   quality_3   747972 non-null  int64  
 7   quality_4   747972 non-null  int64  
 8   quality_5   747972 non-null  object 
 9   quality_6   747972 non-null  int64  
 10  quality_7   747972 non-null  object 
 11  quality_8   747972 non-null  object 
 12  quality_9   747972 non-null  object 
 13  quality_10  747972 non-null  object 
 14  quality_11  747972 non-null  int64  
 15  quality_12  747972 non-null  int64  
dtypes: float64(2), int64(7), object(7)
memory usage: 91.3+ MB


None

train data change column type
column quality_5 is object. change object to float
column quality_7 is object. change object to float
column quality_8 is object. change object to float
column quality_9 is object. change object to float
column quality_10 is object. change object to float
done..
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       828624 non-null  object 
 3   quality_0   828624 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   828624 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828624 non-null  float64
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  float64
 11  quality_8   828624 non-null  float64
 12  qua

None

--------------------------------------
--------------------------------------
--------------------------------------
test data change column type
column quality_1 is object. change object to float
column quality_5 is object. change object to float
column quality_7 is object. change object to float
column quality_8 is object. change object to float
column quality_9 is object. change object to float
column quality_10 is object. change object to float
done..
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        747972 non-null  int64  
 1   user_id     747972 non-null  int64  
 2   fwver       747972 non-null  object 
 3   quality_0   747972 non-null  float64
 4   quality_1   747972 non-null  float64
 5   quality_2   747972 non-null  float64
 6   quality_3   747972 non-null  int64  
 7   quality_4   747972 non-null  int64  
 8   qual

None

#### 1.2.2 merge quality data for user_id

In [ ]:
qual_list = list(train_qual.columns)[3:]
qual_list

#퀄리티 데이터 중 값이 -1인 항목은 모두 0으로 치환
for li in qual_list:
  train_qual[li] = train_qual[li].where(train_qual[li] > 1, 0)
  test_qual[li] = test_qual[li].where(test_qual[li] > 1, 0)
train_qual



,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,4.0,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,4.0,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,4.0,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,4.0,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,4.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,17.0,0,0
828620,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,17.0,0,0
828621,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3.0,0,0.0,0.0,0.0,17.0,0,0
828622,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,17.0,0,0


In [ ]:
"""
user_id 기준으로 퀄리티 데이터 sum
"""

# train set
train_qual.drop(['time','fwver'], axis=1, inplace = True)
tmp_train_qual = train_qual.groupby('user_id').sum().reset_index()
tmp_col = list(train_qual.columns)

train_user_number = 15000
train_user_id_min = 10000
train_user_id_max = 24999
qual_list = tmp_train_qual[tmp_col].values
qual = np.zeros((train_user_number,13))
for q_data  in tqdm(qual_list):
  index_id = int(q_data[0] - train_user_id_min)
  qual[index_id] = q_data[1:]
# qual

user_range = list(range(train_user_id_min,train_user_id_max+1))
qual_df = pd.DataFrame(qual.tolist()).add_prefix('quality_')
# qual_df['user_id'] = user_range
# qual_df = qual_df[tmp_col]
display(qual_df)
qual_df.to_csv(f"{SAVEPATH}/20210129_train_qual_data_mitozero.csv", index = False)



# test set
test_qual.drop(['time','fwver'], axis=1, inplace = True)
tmp_test_qual = test_qual.groupby('user_id').sum().reset_index()
tmp_col = list(test_qual.columns)

test_user_number = 14999
test_user_id_min = 30000
test_user_id_max = 44998

test_qual_list = tmp_test_qual[tmp_col].values
test_qual = np.zeros((test_user_number,13))
for q_data  in tqdm(test_qual_list):
  index_id = int(q_data[0] - test_user_id_min)
  test_qual[index_id] = q_data[1:]
# qual

test_user_range = list(range(test_user_id_min,test_user_id_max+1))
test_qual_df = pd.DataFrame(test_qual.tolist()).add_prefix('quality_')
# test_qual_df['user_id'] = test_user_range
# test_qual_df = test_qual_df[tmp_col]
display(test_qual_df)
test_qual_df.to_csv(f"{SAVEPATH}/20210129_test_qual_data_mitozero.csv", index = False)


100%|██████████| 8281/8281 [00:00<00:00, 409433.13it/s]


,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,27.0,44.0,552.0,0.0,0.0,372.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,87.0,1044.0,0.0,0.0,36.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,228.0,0.0,0.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


100%|██████████| 8268/8268 [00:00<00:00, 404070.07it/s]


,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,60.0,0.0,0.0,24.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,2560.0,0.0,0.0,0.0,0.0,30756.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,133.0,0.0,0.0,0.0,0.0,1596.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,8.0,220.0,2640.0,0.0,0.0,120.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,33.0,396.0,0.0,0.0,24.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
14995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,444.0,0.0,0.0
14997,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,156.0,0.0,0.0


#### 1.2.4 merge quality(-1) data for user_id

In [44]:
qual_list = list(train_qual.columns)[3:]
qual_list

#퀄리티 데이터 중 값이 -1인 항목은 모두 0으로 치환
for li in qual_list:
  # 0 보다 큰 값들 전부 0으로 치환
  train_qual[li] = train_qual[li].where(train_qual[li] < 1, 0)
  train_qual[li] = train_qual[li].replace(-1,1)

  # -1인 값들 1로 치환
  test_qual[li] = test_qual[li].where(test_qual[li] < 1, 0)
  test_qual[li] = test_qual[li].replace(-1,1)


train_qual.drop(['time','fwver'], axis=1, inplace = True)
test_qual.drop(['time','fwver'], axis=1, inplace = True)

train_qual = train_qual.groupby('user_id').sum().reset_index()
test_qual = test_qual.groupby('user_id').sum().reset_index()

# train data 전처리
print('train data...',end='')
train_user_number = 15000
train_user_id_min = 10000
train_user_id_max = 24999
qual_list = train_qual[list(train_qual.columns)].values
qual = np.zeros((train_user_number,13))
for q_data  in tqdm(qual_list):
  index_id = int(q_data[0] - train_user_id_min)
  qual[index_id] = q_data[1:]

train_qual = pd.DataFrame(qual).add_prefix(f'quality_minus_')
train_qual.to_csv(f"{SAVEPATH}/20210203_train_qual_data_minus.csv", index = False)
print("save data done")


# test data 전처리
test_user_number = 14999
test_user_id_min = 30000
test_user_id_max = 44998
test_qual_list = test_qual[list(test_qual.columns)].values
test_qual = np.zeros((test_user_number,13))
for q_data  in tqdm(test_qual_list):
  index_id = int(q_data[0] - test_user_id_min)
  test_qual[index_id] = q_data[1:]
test_qual = pd.DataFrame(test_qual).add_prefix(f'quality_minus_')
test_qual.to_csv(f"{SAVEPATH}/20210203_test_qual_data_minus.csv", index = False)
print("save data done")

  0%|          | 0/8268 [00:00<?, ?it/s]

train data...save data done


100%|██████████| 8268/8268 [00:00<00:00, 338716.82it/s]


save data done


#### 1.2.3 merge datetime data for user_id

In [52]:
def make_weekday_df(df, err_or_qual):
  """
  df = train or test 데이터프레임
  err_or_qual = 에러 데이터인 경우 'err', 퀄리티 데이터인 경우 'qual' 로 표기
  """

  df_datetime = df[['user_id','time']]

  # 주 = [월 화 수 목 금 토 일]
  print("create datetime...",end='')
  df_datetime['datetime'] = df_datetime['time'].apply(make_datetime)
  print('done!')
  # print("create month")
  # df_datetime['month'] = df_datetime['datetime'].map(lambda x : x.month)
  # print("create day")
  # df_datetime['day'] = df_datetime['datetime'].map(lambda x : x.day)
  # print("create hour")
  # df_datetime['hour'] = df_datetime.loc[:, 'datetime'].dt.time
  print("create weekday...",end='')
  df_datetime['weekday'] = df_datetime['datetime'].map(lambda x : x.weekday())
  # display(df_datetime)
  print('done!')

  #주만 따로 분배
  print("merge weekday...",end='')
  df_weekday = pd.get_dummies(df_datetime['weekday'])

  #주 별로 나눈 후 groupby sum
  df_weekday = pd.get_dummies(df_datetime['weekday']).add_prefix(f'{err_or_qual}_week_')
  df_weekday['user_id'] = df['user_id']
  df_weekday = df_weekday.groupby('user_id').sum().reset_index()
  print('done!')
  print(df_weekday)

  return df_weekday

In [53]:
# train data
train_qual_weekday = make_weekday_df(train_qual, 'qual')

# train data 전처리
train_user_number = 15000
train_user_id_min = 10000
train_user_id_max = 24999
qual_list = train_qual_weekday[list(train_qual_weekday.columns)].values
qual = np.zeros((train_user_number,7))
for q_data  in tqdm(qual_list):
  index_id = int(q_data[0] - train_user_id_min)
  qual[index_id] = q_data[1:]

train_qual_weekday = pd.DataFrame(qual, columns=list(train_qual_weekday.columns)[1:])
train_qual_weekday.to_csv(f"{SAVEPATH}/20210203_train_qual_weekday_data.csv", index = False)
print("save data done")



# test data
test_qual_weekday = make_weekday_df(test_qual, 'qual')

# test data 전처리
test_user_number = 14999
test_user_id_min = 30000
test_user_id_max = 44998
test_qual_list = test_qual_weekday[list(test_qual_weekday.columns)].values
test_qual = np.zeros((test_user_number,7))
for q_data  in tqdm(test_qual_list):
  index_id = int(q_data[0] - test_user_id_min)
  test_qual[index_id] = q_data[1:]
test_qual_weekday = pd.DataFrame(test_qual, columns=list(test_qual_weekday.columns)[1:])
test_qual_weekday.to_csv(f"{SAVEPATH}/20210203_test_qual_weekday_data.csv", index = False)
print("save data done")


create datetime...done!
create weekday...

100%|██████████| 8281/8281 [00:00<00:00, 379625.01it/s]

done!
merge weekday...done!
      user_id  qual_week_0  qual_week_1  ...  qual_week_4  qual_week_5  qual_week_6
0       10000         12.0          0.0  ...          0.0          0.0         12.0
1       10002         24.0          0.0  ...         12.0          0.0         12.0
2       10004         12.0          0.0  ...          0.0          0.0          0.0
3       10005          0.0          0.0  ...          0.0          0.0          0.0
4       10006         12.0          0.0  ...         12.0          0.0          0.0
...       ...          ...          ...  ...          ...          ...          ...
8276    24990          0.0          0.0  ...          0.0         12.0         12.0
8277    24992         12.0          0.0  ...          0.0          0.0         24.0
8278    24993         12.0          0.0  ...          0.0          0.0         12.0
8279    24995          0.0          0.0  ...          0.0         12.0         12.0
8280    24997          0.0         12.0  ...    

save data done
create datetime...done!
create weekday...

100%|██████████| 8268/8268 [00:00<00:00, 405028.09it/s]

done!
merge weekday...done!
      user_id  qual_week_0  qual_week_1  ...  qual_week_4  qual_week_5  qual_week_6
0       30000          0.0          0.0  ...          0.0         12.0          0.0
1       30001         12.0          0.0  ...          0.0          0.0          0.0
2       30002         12.0         12.0  ...         60.0          0.0         12.0
3       30003         24.0         24.0  ...          0.0          0.0          0.0
4       30004          0.0          0.0  ...          0.0         12.0          0.0
...       ...          ...          ...  ...          ...          ...          ...
8263    44990          0.0          0.0  ...          0.0         12.0         12.0
8264    44993          0.0          0.0  ...          0.0          0.0         12.0
8265    44994          0.0          0.0  ...          0.0          0.0          0.0
8266    44996         12.0         24.0  ...         12.0         24.0         12.0
8267    44997         12.0          0.0  ...    

#### 1.2.4 merge datetime_time data for user_id

In [ ]:
def make_hours_df(df, err_or_qual):
  """
  요일별(월(0)~일(6))로 데이터를 병합해주는 함수

  df = train or test 데이터프레임
  err_or_qual = 에러 데이터인 경우 'err', 퀄리티 데이터인 경우 'qual' 로 표기
  """

  df_datetime = df[['user_id','time']]

  # 주 = [월 화 수 목 금 토 일]
  print("create datetime...",end='')
  df_datetime['datetime'] = df_datetime['time'].apply(make_datetime)
  print('done!')
  print("create hours...",end='')
  df_datetime['hours'] = df_datetime['datetime'].map(lambda x : x.time().hour)
  # display(df_datetime)
  print('done!')

  #주만 따로 분배
  print("merge hours...",end='')
  df_hours = pd.get_dummies(df_datetime['hours'])

  #주 별로 나눈 후 groupby sum
  df_hours = pd.get_dummies(df_datetime['hours']).add_prefix(f'{err_or_qual}_hours_')
  df_hours['user_id'] = df['user_id']
  df_hours = df_hours.groupby('user_id').sum().reset_index()
  print('done!')
  print(df_hours)

  return df_hours

In [49]:
# train data
train_qual_hours = make_hours_df(train_qual, 'qual')

# train data 전처리
train_user_number = 15000
train_user_id_min = 10000
train_user_id_max = 24999
qual_list = train_qual_hours[list(train_qual_hours.columns)].values
qual = np.zeros((train_user_number,24))
for q_data  in tqdm(qual_list):
  index_id = int(q_data[0] - train_user_id_min)
  qual[index_id] = q_data[1:]

train_qual_hours = pd.DataFrame(qual, columns=list(train_qual_hours.columns)[1:])
train_qual_hours.to_csv(f"{SAVEPATH}/20210203_train_qual_hours_data.csv", index = False)
print("save data done")



# test data
test_qual_hours = make_hours_df(test_qual, 'qual')

# test data 전처리
test_user_number = 14999
test_user_id_min = 30000
test_user_id_max = 44998
test_qual_list = test_qual_hours[list(test_qual_hours.columns)].values
test_qual = np.zeros((test_user_number,24))
for q_data  in tqdm(test_qual_list):
  index_id = int(q_data[0] - test_user_id_min)
  test_qual[index_id] = q_data[1:]
test_qual_hours = pd.DataFrame(test_qual, columns=list(test_qual_hours.columns)[1:])
test_qual_hours.to_csv(f"{SAVEPATH}/20210203_test_qual_hours_data.csv", index = False)
print("save data done")


create datetime...done!
create hours...done!
merge hours...

100%|██████████| 8281/8281 [00:00<00:00, 417640.01it/s]

done!
      user_id  qual_hours_0  ...  qual_hours_22  qual_hours_23
0       10000           0.0  ...            0.0            0.0
1       10002          12.0  ...            0.0            0.0
2       10004           0.0  ...            0.0           12.0
3       10005           0.0  ...            0.0            0.0
4       10006           0.0  ...            0.0            0.0
...       ...           ...  ...            ...            ...
8276    24990           0.0  ...            0.0            0.0
8277    24992           0.0  ...            0.0           12.0
8278    24993           0.0  ...            0.0            0.0
8279    24995          12.0  ...            0.0            0.0
8280    24997           0.0  ...            0.0           12.0

[8281 rows x 25 columns]


save data done
create datetime...done!
create hours...done!
merge hours...

100%|██████████| 8268/8268 [00:00<00:00, 375996.20it/s]

done!
      user_id  qual_hours_0  ...  qual_hours_22  qual_hours_23
0       30000           0.0  ...            0.0            0.0
1       30001           0.0  ...            0.0            0.0
2       30002          24.0  ...            0.0            0.0
3       30003           0.0  ...            0.0            0.0
4       30004           0.0  ...            0.0            0.0
...       ...           ...  ...            ...            ...
8263    44990           0.0  ...            0.0            0.0
8264    44993           0.0  ...           12.0            0.0
8265    44994           0.0  ...            0.0            0.0
8266    44996          12.0  ...           12.0           24.0
8267    44997           0.0  ...            0.0            0.0

[8268 rows x 25 columns]


save data done


### 1.3 all data merge

In [69]:
"""
 학습에 사용되는 모든 컬럼
"""
all_data_col_err = ['err0', 'err1', 'err2', 'err3', 'err4', 'err5', 'err6', 'err7', 'err8', 'err9', 'err10', 
                    'err11', 'err12', 'err13', 'err14', 'err15', 'err16', 'err17', 'err18', 'err19', 'err20', 
                    'err21', 'err22', 'err23', 'err24', 'err25', 'err26', 'err27', 'err28', 'err29', 'err30', 
                    'err31', 'err32', 'err33', 'err34', 'err35', 'err36', 'err37', 'err38', 'err39', 'err40', 'err41']
# all_data_col_errcode = ['A','B','C','D','UNKNOWN']
all_data_col_errcode = ['A','B']
all_data_col_err_weekday = ['err_week_0', 'err_week_1', 'err_week_2', 'err_week_3', 'err_week_4', 'err_week_5', 'err_week_6']
all_data_col_err_hours = ['err_hours_0', 'err_hours_1', 'err_hours_2', 'err_hours_3', 'err_hours_4', 'err_hours_5', 'err_hours_6', 
                          'err_hours_7', 'err_hours_8', 'err_hours_9', 'err_hours_10', 'err_hours_11', 'err_hours_12', 'err_hours_13', 
                          'err_hours_14', 'err_hours_15', 'err_hours_16', 'err_hours_17', 'err_hours_18', 'err_hours_19', 'err_hours_20', 
                          'err_hours_21', 'err_hours_22', 'err_hours_23']

all_data_col_model = ['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'model_7', 'model_8']
all_data_col_fwver = ['03.11.1141', '03.11.1149', '03.11.1167', '04.16.2641', '04.16.3345', '04.16.3439', 
                      '04.16.3553', '04.16.3569', '04.16.3571', '04.22.1170', '04.22.1442', '04.22.1448', 
                      '04.22.1478', '04.22.1608', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', 
                      '04.22.1772', '04.22.1778', '04.33.1095', '04.33.1125', '04.33.1149', '04.33.1171', 
                      '04.33.1185', '04.33.1261', '04.73.2237', '04.73.2569', '04.73.2571', '04.73.2577', 
                      '04.82.1684', '04.82.1730', '04.82.1778', '05.15.2090', '05.15.2092', '05.15.2114', 
                      '05.15.2120', '05.15.2122', '05.15.2138', '05.15.3104', '05.66.3237', '05.66.3571', 
                      '10', '10.22.1770', '10.22.1780', '8.5.3']


all_data_col_qual = ['quality_0', 'quality_1', 'quality_2', 'quality_3', 'quality_4', 'quality_5', 'quality_6', 
                     'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
all_data_col_qual_weekday = ['qual_week_0', 'qual_week_1', 'qual_week_2', 'qual_week_3', 'qual_week_4', 'qual_week_5', 'qual_week_6']
all_data_col_qual_hours = ['qual_hours_0', 'qual_hours_1', 'qual_hours_2', 'qual_hours_3', 'qual_hours_4', 'qual_hours_5', 'qual_hours_6', 
                           'qual_hours_7', 'qual_hours_8', 'qual_hours_9', 'qual_hours_10', 'qual_hours_11', 'qual_hours_12', 'qual_hours_13', 
                           'qual_hours_14', 'qual_hours_15', 'qual_hours_16', 'qual_hours_17', 'qual_hours_18', 'qual_hours_19', 'qual_hours_20', 
                           'qual_hours_21', 'qual_hours_22', 'qual_hours_23']


all_data_columns = ['user_id'] + all_data_col_err + all_data_col_errcode + all_data_col_model + all_data_col_fwver + all_data_col_qual

In [70]:
train_errtype_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210127_train_errtype_data.csv')
# train_errcode_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210131_train_errcode_data_2.csv')
train_modelnm_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_train_modelnm_data.csv')
train_fwver_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210127_train_fwver_data.csv')
train_qual_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210129_train_qual_data_mitozero.csv')
train_qual_minus_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_train_qual_data_minus.csv')
train_err_week_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210201_train_err_weekday_data.csv')
train_err_hours_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_train_err_hours_data.csv')
train_qual_week_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_train_qual_weekday_data.csv')
train_qual_hours_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_train_qual_hours_data.csv')
train_prob_df = pd.read_csv('/content/data/train_problem_data.csv')

test_errtype_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210127_test_errtype_data.csv')
# test_errcode_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210131_test_errcode_data_2.csv')
test_modelnm_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_test_modelnm_data.csv')
test_fwver_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210127_test_fwver_data.csv')
test_qual_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210129_test_qual_data_mitozero.csv')
test_qual_minus_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_test_qual_data_minus.csv')
test_err_week_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210201_test_err_weekday_data.csv')
test_err_hours_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_test_err_hours_data.csv')
test_qual_week_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_test_qual_weekday_data.csv')
test_qual_hours_df =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_test_qual_hours_data.csv')


In [71]:
train_user_number = 15000
train_user_id_min = 10000
train_user_id_max = 24999
train_user_id = list(range(train_user_id_min,train_user_id_max+1))
print(len(train_user_id))

problem = np.zeros(15000)
problem[train_prob_df.user_id.unique()-10000] = 1

test_user_number = 14999
test_user_id_min = 30000
test_user_id_max = 44998
test_user_id = list(range(test_user_id_min,test_user_id_max+1))
print(len(test_user_id))


15000
14999


In [72]:
train_df = pd.DataFrame({'user_id': train_user_id})
train_df = train_df.join(train_errtype_df)
# train_df = train_df.join(train_errcode_df)
train_df = train_df.join(train_modelnm_df)
train_df = train_df.join(train_fwver_df)
train_df = train_df.join(train_qual_df)
train_df = train_df.join(train_qual_minus_df)
train_df = train_df.join(train_err_week_df)
train_df = train_df.join(train_qual_week_df)
train_df = train_df.join(train_err_hours_df)
train_df = train_df.join(train_qual_hours_df)
train_df['problem'] = problem
train_df.to_csv(f"{SAVEPATH}/20210203_train_data_merge_2.csv", index = False)
train_df

,user_id,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,...,err_hours_9,err_hours_10,err_hours_11,err_hours_12,err_hours_13,err_hours_14,err_hours_15,err_hours_16,err_hours_17,err_hours_18,err_hours_19,err_hours_20,err_hours_21,err_hours_22,err_hours_23,qual_hours_0,qual_hours_1,qual_hours_2,qual_hours_3,qual_hours_4,qual_hours_5,qual_hours_6,qual_hours_7,qual_hours_8,qual_hours_9,qual_hours_10,qual_hours_11,qual_hours_12,qual_hours_13,qual_hours_14,qual_hours_15,qual_hours_16,qual_hours_17,qual_hours_18,qual_hours_19,qual_hours_20,qual_hours_21,qual_hours_22,qual_hours_23,problem
0,10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,8.0,5.0,5.0,0.0,10.0,9.0,6.0,15.0,19.0,12.0,0.0,10.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
1,10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,...,57.0,28.0,83.0,160.0,169.0,183.0,193.0,210.0,153.0,85.0,230.0,176.0,258.0,70.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,20.0,8.0,0.0,6.0,4.0,27.0,5.0,6.0,5.0,28.0,24.0,30.0,13.0,10.0,12.0,36.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,...,4.0,12.0,1.0,3.0,0.0,0.0,8.0,3.0,5.0,5.0,19.0,39.0,30.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,...,30.0,10.0,8.0,11.0,36.0,8.0,7.0,18.0,22.0,69.0,111.0,43.0,31.0,70.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,6.0,0.0,12.0,6.0,0.0,14.0,11.0,11.0,20.0,4.0,14.0,18.0,16.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0
14996,24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,...,56.0,9.0,16.0,25.0,14.0,8.0,8.0,11.0,16.0,35.0,19.0,65.0,39.0,62.0,41.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1.0
14998,24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,

In [73]:
test_df = pd.DataFrame({'user_id': test_user_id})
test_df = test_df.join(test_errtype_df)
# test_df = test_df.join(test_errcode_df)
test_df = test_df.join(test_modelnm_df)
test_df = test_df.join(test_fwver_df)
test_df = test_df.join(test_qual_df)
test_df = test_df.join(test_qual_minus_df)
test_df = test_df.join(test_err_week_df)
test_df = test_df.join(test_qual_week_df)
test_df = test_df.join(test_err_hours_df)
test_df = test_df.join(test_qual_hours_df)
test_df.to_csv(f"{SAVEPATH}/20210203_test_data_merge_2.csv", index = False)
test_df

,user_id,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,...,err_hours_8,err_hours_9,err_hours_10,err_hours_11,err_hours_12,err_hours_13,err_hours_14,err_hours_15,err_hours_16,err_hours_17,err_hours_18,err_hours_19,err_hours_20,err_hours_21,err_hours_22,err_hours_23,qual_hours_0,qual_hours_1,qual_hours_2,qual_hours_3,qual_hours_4,qual_hours_5,qual_hours_6,qual_hours_7,qual_hours_8,qual_hours_9,qual_hours_10,qual_hours_11,qual_hours_12,qual_hours_13,qual_hours_14,qual_hours_15,qual_hours_16,qual_hours_17,qual_hours_18,qual_hours_19,qual_hours_20,qual_hours_21,qual_hours_22,qual_hours_23
0,30000,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,...,12.0,123.0,30.0,11.0,32.0,31.0,105.0,284.0,157.0,133.0,143.0,522.0,475.0,97.0,113.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0
1,30001,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.0,5.0,20.0,12.0,25.0,1.0,9.0,5.0,4.0,14.0,9.0,24.0,12.0,8.0,9.0,19.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
2,30002,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,...,37.0,26.0,31.0,55.0,26.0,34.0,32.0,47.0,58.0,63.0,92.0,38.0,10.0,23.0,14.0,16.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,24.0,0.0,36.0,0.0,12.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
3,30003,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,...,14.0,13.0,6.0,4.0,23.0,14.0,6.0,11.0,29.0,13.0,10.0,31.0,13.0,17.0,20.0,10.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0
4,30004,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,...,99.0,43.0,11.0,11.0,5.0,37.0,66.0,55.0,34.0,32.0,31.0,27.0,32.0,24.0,13.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,179.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,13.0,34.0,78.0,68.0,36.0,101.0,37.0,89.0,72.0,97.0,46.0,62.0,128.0,88.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14995,44995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,183.0,0.0,14.0,1.0,0.0,1.0,1.0,0.0,0.0,...,3.0,12.0,27.0,35.0,46.0,24.0,20.0,12.0,14.0,14.0,29.0,27.0,59.0,18.0,32.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,44996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,0.0,0.0,0.0,138.0,165.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,163.0,0.0,12.0,1118.0,0.0,1.0,1.0,1.0,0.0,...,21.0,28.0,174.0,56.0,110.0,27.0,29.0,113.0,183.0,103.0,103.0,273.0,129.0,164.0,189.0,50.0,12.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,24.0
14997,44997,0.0,0.0